# Entropy estimation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import math
from tqdm import tqdm
from scipy.optimize import fsolve

In [ ]:
lag = 16
stats_count = lag + 4
output_dim = 1
input_data_dim = 20
training_set_ratio = 0.8
#batch_size

## Data-Preprocessing

In [ ]:
import math

training_set_ratio = 0.8

with open('random_files/dev-random.bin','rb') as fr:
    fr.seek(0,2)
    file_size = fr.tell()
    training_set_size = math.floor(0.8*file_size)
    
    
    fr.seek(0,0)
    data_train = fr.read(training_set_size)
    data_test = fr.read()

In [ ]:
byte_string = "{0:08b}".format(int.from_bytes(data_train,'big'))

In [ ]:
X_data_train = [float(char) for char in byte_string[:-((training_set_size*8)%(input_data_dim+1))]]

In [ ]:
Y_train = X_data_train[20::21]

In [ ]:
del X_data_train[20::21]

In [ ]:
X_data_train = torch.tensor(X_data_train)

In [ ]:
X_data_train

In [ ]:
X_data_train = X_data_train.view(((training_set_size*8)//(input_data_dim+1)), input_data_dim)

In [ ]:
X_data_train

In [ ]:
Y_train = torch.tensor(Y_train)

In [ ]:
Y_train = Y_train.view(((training_set_size*8)//(input_data_dim+1)), 1)

In [ ]:
Y_train

In [ ]:
byte_string = "{0:08b}".format(int.from_bytes(data_test,'big'))

In [ ]:
len(byte_string)

In [ ]:
X_data_test = [float(char) for char in byte_string[:-((((file_size-training_set_size)*8)%(input_data_dim+1))-1)]]

In [ ]:
len(X_data_test)

In [ ]:
Y_test = X_data_test[20::21]

In [ ]:
del X_data_test[20::21]
X_data_test = torch.tensor(X_data_test)
X_data_test = X_data_test.view((((file_size-training_set_size)*8)//(input_data_dim+1)), input_data_dim)
Y_test = torch.tensor(Y_test)
Y_test = Y_test.view((((file_size-training_set_size)*8)//(input_data_dim+1)), 1)

## Neural Network

In [ ]:
class Neural_Network(nn.Module):
    def __init__(self, stats_count, input_data_dim, num_classes):
        super(Neural_Network, self).__init__()
        self.fc1 = nn.Linear(stats_count, 20)
        self.fc2 = nn.Linear(20, 1)
        self.fc3 = nn.Linear(input_data_dim+1, 128)
        self.fc4 = nn.Linear(128, 20)
        self.fc5 = nn.Linear(20, num_classes)
    
    
    def forward(self, stats, input_data):
        x = F.relu(self.fc1(stats))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(torch.cat((input_data, x), dim=1)))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))        #softmax
        return x

model = Neural_Network(stats_count, input_data_dim, output_dim)
print(model)

In [ ]:
loss_function = nn.L1Loss()
total_epochs = 20000
learning_rate = 0.001    #update the equation
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
X_stats_train = torch.randn(((training_set_size*8)//(input_data_dim+1)), 20)

In [ ]:
X_stats_train

In [ ]:
output = model(X_stats_train,X_data_train)

In [ ]:
for epoch in tqdm(range(total_epochs)):
    model.zero_grad()
    output = model(X_stats_train, X_data_train)
    #print(output)
    
    loss = loss_function(output, Y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    #print(loss)
    

In [ ]:
X_stats_test = torch.randn((((file_size-training_set_size)*8)//(input_data_dim+1)), 20)

In [ ]:
correct = 0
total = 0
longest_run = 0
with torch.no_grad():
    #for data in testset:
    output = model(X_stats_test, X_data_test)
    run = 0
    for idx, i in enumerate(output):
        #print(float(output[idx]))
        #print(Y_test[idx])
        if math.floor(float(output[idx])+0.5) == Y_test[idx]:
            run = run + 1
            correct = correct+1
        else:
            longest_run = max(longest_run, run)
            run = 0
        total = total+1
    
    loss = loss_function(output, Y_test)
    #print(output)
    print(loss)
    print(correct)
    print(total)
    print(longest_run)

## Entropy calculation

In [ ]:
n = total       #number of bits produced
c = correct       #number of correct bits
l = longest_run        #longest run

In [ ]:
prediction_global = c/n

if prediction_global == 0 :
    prediction_global_normalized = 1 - (0.01**(1/n))

else:
    prediction_global_normalized = min(1, prediction_global+2.579*(((prediction_global*(1-prediction_global))/(n-1))**(1/2))) #99% confidence

In [ ]:
def local_predictor(p, *args):
    
    l, n, precision = args
    
    
    q = 1-p
    x = 1+q*(p**l)
    
    
    for i in range(int(precision)):
        x = x + (((l+1)**(i+1))*((q*(p**l))**(i+2)))     #check equation
    
    return ((1-p*x)/((l+2-(l+1)*x)*q*(x**(n+1))))-0.99


In [ ]:
predict =  fsolve(local_predictor, 0.5, (l, n, 0))

print(predict)
print(local_predictor(predict, l, n, 0))

In [ ]:
precision = 0
efselon = 1e-5


predict =  fsolve(local_predictor, 0.5, (l, n, precision))
precision = precision+1

predict_new = fsolve(local_predictor, 0.5, (l, n, precision))


while abs(predict-predict_new)>efselon:
    precision = precision+1
    predict = predict_new
    predict_new = fsolve(local_predictor, 0.5, (l, n, precision))


prediction_local = predict_new

In [ ]:
min_Entropy = -math.log(max(prediction_global_normalized, prediction_local),2)

In [ ]:
min_Entropy